##  Project: Predicting Customer Churn 

In [1]:
#Libraries
import pyodbc
from  dotenv import dotenv_values    #import dotenv_values function from the dotenv package
import pandas as pd
import numpy as np
from scipy import stats
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
#import squarify
#import re
warnings.filterwarnings("ignore")

Creation of environment variables to hide sensitive information and connect remotely to database

## Data Loading

In [2]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [3]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection = pyodbc.connect(connection_string)

In [4]:
# Now create the sql query to get the data is what what you see below. 

query1 = "Select * from  dbo.LP2_Telco_churn_first_3000"

In [5]:
#Run the query wih aid of the connection and convert the target data_1 into a dataframe
data_1 = pd.read_sql(query1, connection)

In [6]:
#Preview of the first five rows of the data_1 DataFrame 
data_1.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,None,DSL,False,...,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,...,True,False,False,False,One year,False,Mailed check,56.950001,1889.500000,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,...,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.150002,True
3,7795-CFOCW,Male,False,False,False,45,False,None,DSL,True,...,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.750000,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.649994,True


In [7]:
#Information summary of data_1
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        3000 non-null   object 
 1   gender            3000 non-null   object 
 2   SeniorCitizen     3000 non-null   bool   
 3   Partner           3000 non-null   bool   
 4   Dependents        3000 non-null   bool   
 5   tenure            3000 non-null   int64  
 6   PhoneService      3000 non-null   bool   
 7   MultipleLines     2731 non-null   object 
 8   InternetService   3000 non-null   object 
 9   OnlineSecurity    2349 non-null   object 
 10  OnlineBackup      2349 non-null   object 
 11  DeviceProtection  2349 non-null   object 
 12  TechSupport       2349 non-null   object 
 13  StreamingTV       2349 non-null   object 
 14  StreamingMovies   2349 non-null   object 
 15  Contract          3000 non-null   object 
 16  PaperlessBilling  3000 non-null   bool   


In [8]:
#Loading of data_2
data_2 =pd.read_csv(r'C:\Users\WORLDAX COMPUTERS\Python Projects\LP_2-Data_Analytics_Project\Telecommunications-Churn-Analysis\Datasets\LP2_Telco-churn-last-2000.csv')
#Preview of first five rows in data_2
data_2.head() 

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5600-PDUJF,Male,0,No,No,6,Yes,No,DSL,No,...,No,Yes,No,No,Month-to-month,Yes,Credit card (automatic),49.50,312.7,No
1,8292-TYSPY,Male,0,No,No,19,Yes,No,DSL,No,...,Yes,Yes,No,No,Month-to-month,Yes,Credit card (automatic),55.00,1046.5,Yes
2,0567-XRHCU,Female,0,Yes,Yes,69,No,No phone service,DSL,Yes,...,Yes,No,No,Yes,Two year,Yes,Credit card (automatic),43.95,2960.1,No
3,1867-BDVFH,Male,0,Yes,Yes,11,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,74.35,834.2,Yes
4,2067-QYTCF,Female,0,Yes,No,64,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,111.15,6953.4,No


In [9]:
#Information summary of Data_2
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043 entries, 0 to 2042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        2043 non-null   object 
 1   gender            2043 non-null   object 
 2   SeniorCitizen     2043 non-null   int64  
 3   Partner           2043 non-null   object 
 4   Dependents        2043 non-null   object 
 5   tenure            2043 non-null   int64  
 6   PhoneService      2043 non-null   object 
 7   MultipleLines     2043 non-null   object 
 8   InternetService   2043 non-null   object 
 9   OnlineSecurity    2043 non-null   object 
 10  OnlineBackup      2043 non-null   object 
 11  DeviceProtection  2043 non-null   object 
 12  TechSupport       2043 non-null   object 
 13  StreamingTV       2043 non-null   object 
 14  StreamingMovies   2043 non-null   object 
 15  Contract          2043 non-null   object 
 16  PaperlessBilling  2043 non-null   object 


Shapes of Dataframes

In [10]:
data_1.shape

(3000, 21)

In [11]:
data_2.shape

(2043, 21)

## Data Cleaning
cleaning of dataframe data_1

In [12]:
#Information summary
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        3000 non-null   object 
 1   gender            3000 non-null   object 
 2   SeniorCitizen     3000 non-null   bool   
 3   Partner           3000 non-null   bool   
 4   Dependents        3000 non-null   bool   
 5   tenure            3000 non-null   int64  
 6   PhoneService      3000 non-null   bool   
 7   MultipleLines     2731 non-null   object 
 8   InternetService   3000 non-null   object 
 9   OnlineSecurity    2349 non-null   object 
 10  OnlineBackup      2349 non-null   object 
 11  DeviceProtection  2349 non-null   object 
 12  TechSupport       2349 non-null   object 
 13  StreamingTV       2349 non-null   object 
 14  StreamingMovies   2349 non-null   object 
 15  Contract          3000 non-null   object 
 16  PaperlessBilling  3000 non-null   bool   


In [13]:
#A preview of data in columns
data_1.describe(include='all')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,3000,3000,3000,3000,3000,3000.000000,3000,2731,3000,2349,...,2349,2349,2349,2349,3000,3000,3000,3000.000000,2995.000000,2999
unique,3000,2,2,2,2,NaN,2,2,3,2,...,2,2,2,2,3,2,4,NaN,NaN,2
top,7590-VHVEG,Male,False,False,False,NaN,True,False,Fiber optic,False,...,False,False,False,True,Month-to-month,True,Electronic check,NaN,NaN,False
freq,1,1537,2525,1538,2070,NaN,2731,1437,1343,1478,...,1296,1476,1190,1199,1641,1776,1004,NaN,NaN,2219
mean,NaN,NaN,NaN,NaN,NaN,32.527333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.347400,2301.278315,NaN
std,NaN,NaN,NaN,NaN,NaN,24.637768,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.137053,2274.987884,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.400000,18.799999,NaN
25%,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.787499,415.250000,NaN
50%,NaN,NaN,NaN,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.900002,1404.650024,NaN
75%,NaN,NaN,NaN,NaN,NaN,56.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.262501,3868.725098,NaN


In [14]:
#Check of unique values in each column
data_1.nunique()

customerID          3000
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          2
InternetService        3
OnlineSecurity         2
OnlineBackup           2
DeviceProtection       2
TechSupport            2
StreamingTV            2
StreamingMovies        2
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1198
TotalCharges        2893
Churn                  2
dtype: int64

Check of Missing Values

In [15]:
# Create a loop to iterate over each column in the DataFrame
print('Count of missing values in columns')
print()
for column in data_1.columns:
    # Use the 'isna()' method on the column to check for missing values
    missin_val = data_1[column].isna()

    # Calculate the sum of missin_val in the column
    missin_val_sum = missin_val.sum()

    # Check if any missin_val exist in the column
    if missin_val_sum > 0:
        print(f'{column}": {missin_val_sum}')
        print('-------' * 3)

Count of missing values in columns

MultipleLines": 269
---------------------
OnlineSecurity": 651
---------------------
OnlineBackup": 651
---------------------
DeviceProtection": 651
---------------------
TechSupport": 651
---------------------
StreamingTV": 651
---------------------
StreamingMovies": 651
---------------------
TotalCharges": 5
---------------------
Churn": 1
---------------------


Cleaning 'MultipleLines' column 

In [16]:
#Unique elements

data_1['MultipleLines'].unique()

array([None, False, True], dtype=object)

In [17]:
#I will fill missing values with 'No phone service' to create third category and avoid inconsistency
#This is because data_2 has 3 unique elements, namely, 'No', 'Yes', 'No phone service'

data_1['MultipleLines'] = data_1['MultipleLines'].fillna('No phone service')
#Were changes effected?
data_1['MultipleLines'].unique()

array(['No phone service', False, True], dtype=object)

Cleaning 'OnlineSecurity' column 

In [18]:
#Check of unique elements

data_1["OnlineSecurity"].unique()

array([False, True, None], dtype=object)

In [19]:
#I will fill missing values with "No internet service" based on domain knowledge
#Whether the customer has online security or not (Yes, No, No Internet)

data_1['OnlineSecurity'] = data_1['OnlineSecurity'].fillna('No internet service')
#Were changes effected?
data_1['OnlineSecurity'].unique()

array([False, True, 'No internet service'], dtype=object)

Cleaning 'OnlineBackup' column 

In [20]:
#Check of unique elements
data_1["OnlineBackup"].unique()

array([True, False, None], dtype=object)

In [21]:
#Whether the customer has online backup or not (Yes,No, No Internet)
#Based on domain knowledge I filled missing values with 'No internet service'

data_1['OnlineBackup'] = data_1['OnlineBackup'].fillna('No internet service')
#Are changes effected?
data_1['OnlineBackup'].unique()

array([True, False, 'No internet service'], dtype=object)

Cleaning 'DeviceProtection' column

In [22]:
#Check of unique elements
data_1["DeviceProtection"].unique()

array([False, True, None], dtype=object)

In [23]:
#Whether the customer has device protection or not (Yes,No, No Internet service)
#I filled missing values with 'No internet service' to achieve consistency

data_1['DeviceProtection'] = data_1['DeviceProtection'].fillna('No internet service')
#Are changes effected? Yeah!
data_1['DeviceProtection'].unique()


array([False, True, 'No internet service'], dtype=object)

Cleaning 'TechSupport' column 


In [24]:
#Check of unique elements
data_1["TechSupport"].unique()

array([False, True, None], dtype=object)

In [25]:
#Whether the customer has Tech Support or not (Yes,No, No Internet)
#I filled missing values with 'No internet service' to achieve consistency

data_1['TechSupport'] = data_1['TechSupport'].fillna('No internet service')
#Are changes effected? Yeah!
data_1['TechSupport'].unique()

array([False, True, 'No internet service'], dtype=object)

Cleaning 'StreamingTV' column 

In [26]:
#Check of unique elements
data_1["StreamingTV"].unique()

array([False, True, None], dtype=object)

In [27]:
#Whether the customer has Streaming TV or not (Yes,No, No Internet)
#I filled missing values with 'No internet service' to achieve consistency

data_1['StreamingTV'] = data_1['StreamingTV'].fillna('No internet service')

#Are changes effected? Yea!
data_1['StreamingTV'].unique()

array([False, True, 'No internet service'], dtype=object)

Cleaning 'StreamingMovies' column 

In [28]:
#Check of unique elements
data_1["StreamingMovies"].unique()

array([False, True, None], dtype=object)

In [29]:
#Whether the customer has Streaming Movies or not (Yes,No, No Internet service)
#I filled missing values with 'No internet service' to achieve consistency

data_1['StreamingMovies'] = data_1['StreamingMovies'].fillna('No internet service')

#Are changes effected? Yea!
data_1['StreamingMovies'].unique()

array([False, True, 'No internet service'], dtype=object)

Cleaning 'TotalCharges' column 

In [30]:
#Check of unique values
data_1["TotalCharges"].isna().sum()

5

In [31]:
#Measure of central tendency of the column 'TotalCharges'
data_1["TotalCharges"].describe()

count    2995.000000
mean     2301.278315
std      2274.987884
min        18.799999
25%       415.250000
50%      1404.650024
75%      3868.725098
max      8564.750000
Name: TotalCharges, dtype: float64

In [32]:
#Column data is skewed, and has high variability based on the difference between mean and std deviation.
#Check of median value

data_1["TotalCharges"].median()


1404.6500244140625

In [37]:
#I filled missing values with the median because mean is largely affected by outliers 

data_1["TotalCharges"] = data_1["TotalCharges"].fillna(data_1["TotalCharges"].median())

#Any missing values remaining in the column?
data_1["TotalCharges"].isna().sum()

0

Cleaning 'Churn' column

In [40]:
#Check of unique values
data_1["Churn"].unique()

array([False, True, None], dtype=object)

In [43]:
#Count of missing values
#Column has one missing value

data_1["Churn"].isna().sum()

1

In [48]:
#I chose to fill the missing value with the mode value

data_1["Churn"] = data_1["Churn"].fillna(data_1["Churn"].mode().iloc[0])

#Any missing value left?
data_1["Churn"].isna().sum()

0

In [49]:
#A look of the dataframe information after the cleaning process
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        3000 non-null   object 
 1   gender            3000 non-null   object 
 2   SeniorCitizen     3000 non-null   bool   
 3   Partner           3000 non-null   bool   
 4   Dependents        3000 non-null   bool   
 5   tenure            3000 non-null   int64  
 6   PhoneService      3000 non-null   bool   
 7   MultipleLines     3000 non-null   object 
 8   InternetService   3000 non-null   object 
 9   OnlineSecurity    3000 non-null   object 
 10  OnlineBackup      3000 non-null   object 
 11  DeviceProtection  3000 non-null   object 
 12  TechSupport       3000 non-null   object 
 13  StreamingTV       3000 non-null   object 
 14  StreamingMovies   3000 non-null   object 
 15  Contract          3000 non-null   object 
 16  PaperlessBilling  3000 non-null   bool   


In [41]:
data_2["Churn"].unique()

array(['No', 'Yes'], dtype=object)

In [47]:

data_2["Churn"].isna().sum()

0